In [1]:
import os
import numpy as np
import torch
from numpy import random

import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

from pathlib import Path
home = str(Path.home())

# Attacks

In [61]:
factor = 0.8
run_nr = [0,1,2]
mean_tables = []
nr_samples=2000

# datasets = ['cifar10', 'cifar100', 'imagenet']
# models = ['wrn28-10', 'vgg16', 'wrn50-2']
# attacks = ['fgsm', 'bim', 'pgd', 'aa', 'apgd-ce', 'df', 'cw']

datasets = ['cifar10', 'imagenet']
models = ['wrn28-10', 'wrn50-2']
attacks = ['apgd-ce']
# norm = 'l2'
norm = 'linf'

if norm == 'l2':
    eps=['0.1', '0.2', '0.3', '0.4', '0.5']
else:
    eps=['05255', '1255', '2255', '4255', '8255']

for it in run_nr:
    for dataset in datasets:        
        for model in models:
            
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
                
            try:
                for att in attacks:
                    for ep in eps: 
                        base_path = os.path.join(home, 'workspace/multiLID/data/gen/run_{}/'.format(it+1))
                        curr_eps = ""
                        if att in ['fgsm', 'bim', 'pgd', 'aa']:
                            curr_eps = "_" + ep
                        if att in ['apgd-ce']:
                            if dataset in ['imagenet']:
                                curr_eps = "_apgd-ce_{}_".format(norm) + ep
                            else:
                                curr_eps = "_" + ep
                            
                        filename = f"normalos{curr_eps}.pt"
                        
                        nor_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{att}/{filename}"))
                        adv_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{att}/{filename}"))

            except OSError as e:
                print(f"{type(e)}: {e}")

# Table 1

In [53]:
run_nr = [0,1,2]
mean_tables = []
nr_samples=2000

datasets = ['cifar10', 'cifar100', 'imagenet']
models = ['wrn28-10', 'vgg16', 'wrn50-2']
attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
detectors = ['lid', 'multilid']

for it in run_nr:
    for dataset in datasets:        
        if dataset == 'imagenet':
            k=30
        else:
            k=20
        
        for model in models:
            
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
                  
            for detector in detectors:
                try:
                    base_path = os.path.join(home, 'workspace/multiLID/data/extract/run_{}/'.format(it+1))
                    nor_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/fgsm/k{k}/{detector}_normalos_8255.pt"))
                    adv_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/fgsm/k{k}/{detector}_adverlos_8255.pt"))
                    nor_bim   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/bim/k{k}/{detector}_normalos_8255.pt"))
                    adv_bim   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/bim/k{k}/{detector}_adverlos_8255.pt"))
                    nor_pgd   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/pgd/k{k}/{detector}_normalos_8255.pt"))
                    adv_pgd   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/pgd/k{k}/{detector}_adverlos_8255.pt"))
                    nor_aa    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/aa/k{k}/{detector}_normalos_8255.pt"))
                    adv_aa    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/aa/k{k}/{detector}_adverlos_8255.pt"))
                    nor_df    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/df/k{k}/{detector}_normalos.pt"))
                    adv_df    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/df/k{k}/{detector}_adverlos.pt"))
                    nor_cw    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/cw/k{k}/{detector}_normalos.pt"))
                    adv_cw    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/cw/k{k}/{detector}_adverlos.pt"))
                except OSError as e:
                    print(f"{type(e)}: {e}")

# Figure 5

In [6]:
attacks=['apgd-ce']
run_nrs = [1,2,3]
datasets=['cifar10', 'imagenet']
# method='lid'
method='multilid'
lid_ks=['3', '5', '10', '30', '50']
norm = 'l2'
# norm = 'linf'

dict_res_rf = {}
dict_res_lr = {}

if norm == 'l2':
    eps=['0.1', '0.2', '0.3', '0.4', '0.5']
else:
    eps=['05255', '1255', '2255', '4255', '8255']
    
for dataset in datasets:
    if dataset == 'cifar10':
        ds = 'cifar10/wrn28-10'
    else:
        ds = 'imagenet/wrn50-2'
    print(ds)

    table_rf = np.zeros((len(eps), len(lid_ks)))
    table_lr = np.zeros((len(eps), len(lid_ks)))

    for att in attacks:
        for run_nr in run_nrs:
            print("attack", att)
            random_state = np.random.randint(0, 100)
            for i_eps, ep in enumerate(eps):
                for j_eps, lid_k in enumerate(lid_ks):
                    curr_eps = ""
                    if att in ['fgsm', 'bim', 'pgd', 'aa']:
                        curr_eps = "_" + ep
                    if att in ['apgd-ce']:
                        curr_eps = "_apgd-ce_{}_".format(norm) + ep

                    try:
                        base_path = os.path.join(home, 'workspace/multiLID/data/extract/run_{}/'.format(run_nr))
                        clean     = torch.load(os.path.join(base_path.replace('run_1', 'run_{}'.format(run_nr)), ds + '/{}/{}/k{}/{}_normalos{}.pt'.format(method, att, lid_k, method, curr_eps)) ).numpy()
                        chars_adv = torch.load(os.path.join(base_path.replace('run_1', 'run_{}'.format(run_nr)), ds + '/{}/{}/k{}/{}_adverlos{}.pt'.format(method, att, lid_k, method, curr_eps)) ).numpy()
                    except OSError as e:
                        print(f"{type(e)}: {e}")

cifar10/wrn28-10
attack apgd-ce
attack apgd-ce
attack apgd-ce
imagenet/wrn50-2
attack apgd-ce
attack apgd-ce
attack apgd-ce
